In [0]:
%python
%pip install beautifulsoup4

In [0]:
from pyspark.sql.types import StringType

In [0]:
import requests
from bs4 import BeautifulSoup

BASE_URL_CNPJ = "https://arquivos.receitafederal.gov.br/dados/cnpj/dados_abertos_cnpj/"
BASE_URL_REGIME = "https://arquivos.receitafederal.gov.br/dados/cnpj/regime_tributario/"

def getlinks(baseurl):

    res = requests.get(baseurl)
    soup = BeautifulSoup(res.text, "html.parser")

    todos_links = []

    arquivos_raiz = [a["href"] for a in soup.find_all("a") if a["href"].endswith(".zip")]
    for arq in arquivos_raiz:
        todos_links.append(baseurl + arq)

    subpastas = [a["href"] for a in soup.find_all("a") if a["href"].endswith("/") and a["href"] != "../"]

    for sub in subpastas:
        url_sub = baseurl + sub
        res_sub = requests.get(url_sub)
        soup_sub = BeautifulSoup(res_sub.text, "html.parser")

        arquivos = [a["href"] for a in soup_sub.find_all("a") if a["href"].endswith(".zip")]

        for arq in arquivos:
            link_completo = url_sub + arq
            todos_links.append(link_completo)

    for link in todos_links:
        print(link)

    return todos_links


In [0]:
cnpj = getlinks(BASE_URL_CNPJ)

In [0]:
regime = getlinks(BASE_URL_REGIME)

In [0]:
df_regime = spark.createDataFrame(regime, StringType())
df_cnpj = spark.createDataFrame(cnpj, StringType())

df_regime.write\
    .mode("overwrite")\
    .format("json")\
    .save("/Volumes/workspace/discovery/discovery/links/regime")

df_cnpj.write\
    .mode("overwrite")\
    .format("json")\
    .save("/Volumes/workspace/discovery/discovery/links/cnpj")